In [ ]:
from inspect_ai import eval, Task, task
from inspect_ai.dataset import Sample
from inspect_ai.scorer import Score, Target, accuracy, scorer
from inspect_ai.solver import Plan, TaskState, generate, use_tools, basic_agent
from inspect_ai.tool import bash
from inspect_ai.util import sandbox

In [ ]:
@scorer(metrics=[accuracy()])
def check_file_exists():
    async def score(state: TaskState, target: Target):
        try:
            output = await sandbox().exec('ls')
            print(output)
            _ = await sandbox().read_file(target.text)
            exists = True
        except FileNotFoundError:
            exists = False
        return Score(value=1 if exists else 0)

    return score

In [ ]:
agent = basic_agent(tools=[bash()])

In [ ]:
@task
def hello_world_challenge() -> Task:
    return Task(
        dataset=[
            Sample(
                input=(
                    "Create a file called `hello-world.txt`. "
                    "When you have finished, submit the string 'submit'."  # Without this, the agent is liable to loop until it hits its limits.
                ),
                target="hello-world.txt",
            )
        ],
        # solver=[use_tools([bash()]), generate()],
        solver=agent,
        sandbox="local",
        scorer=check_file_exists(),
    )

In [ ]:
eval(hello_world_challenge,
     model="openai/gpt-4o-mini",
     )